# Лабораторна робота №10. Класифiкацiя текстових даних за допомогою нейромереж.

Завданням на цій лабораторній роботі є створити простий класифікатор та класифікувати дані з набору imdb та оцінити його роботу.

### Імпорт бібліотек.

### Завантаження даних.
Запустіть цю клітинку, щоб завантажити дані. У цій лабораторній буде використано весь набір бази даних imdb, або 25000 записів.

In [2]:
from datasets import load_dataset

imdb = load_dataset("imdb")
imdb = imdb['train']

Found cached dataset imdb (C:/Users/simic/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)


  0%|          | 0/3 [00:00<?, ?it/s]

### Створіть датафрейм з словника 'imdb'.

,text,label
0,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,"""I Am Curious: Yellow"" is a risible and preten...",0
2,If only to avoid making this type of film in t...,0
3,This film was probably inspired by Godard's Ma...,0
4,"Oh, brother...after hearing about this ridicul...",0


### Проведіть попередню обробку даних.
Необхідно видалити всі небуквенні символи з набору даних, перевести всі символи у нижній регістр, провести стематизацію слів та видалити стоп слова з відгуків.

['rent curiou yellow video store controversi surround first releas also heard first seiz u custom ever tri enter countri therefor fan film consid controversi realli see br br plot center around young swedish drama student name lena want learn everyth life particular want focu attent make sort documentari averag swede thought certain polit issu vietnam war race issu unit state ask politician ordinari denizen stockholm opinion polit sex drama teacher classmat marri men br br kill curiou yellow year ago consid pornograph realli sex nuditi scene far even not shot like cheapli made porno countrymen mind find shock realiti sex nuditi major stapl swedish cinema even ingmar bergman arguabl answer good old boy john ford sex scene film br br commend filmmak fact sex shown film shown artist purpos rather shock peopl make money shown pornograph theater america curiou yellow good film anyon want studi meat potato pun intend swedish cinema realli film much plot', 'curiou yellow risibl pretenti steam

### Створіть модель Bag of Words.

### Розділ моделі на тренувальні та тестові дані.

### Створіть та натренуйте класифікатор.

GaussianNB()

### Протестуйте роботу моделі на тестових даних.

[[1 1]
 [0 0]
 [1 1]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Оцініть роботу моделі використовуючи матрицю конфузії.

[[2133  363]
 [ 913 1591]]


0.7448

### Створіть свою рекурентну/LSTM мережу, яка здатна класифікувати текст.